# Network to classify images as cats or dogs

In [1]:
# Copying 4000 (2000 of each class) images to training, validation and test directories

import os, shutil

original_dataset_dir = './cats_and_dogs_original'
base_dir = './cats_and_dogs_small'
os.mkdir(base_dir)

In [2]:
# directories for the training, validation and test splits
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

#directories with pictures
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

In [7]:
# copies the first 1,000 cat images to train_cat_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# copies the next 500 cat images to vaidation_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# copies the next 500 cat images to test_cat_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# dogs
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)
    

### Building a network

Using a pretrained convnet

In [ ]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [4]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

Extending the conv_base model by adding Dense layers on top will allow to use data augmentation.

In [5]:
# adding a densely connected classifier on top of the convulational base

from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

# data augmentation (only for train data)

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# image preprocessing

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),  # resize all images to 150x150
        batch_size=20,
        class_mode='binary')  # binary labels

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [8]:
# compiling and training a model

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/30
100/100 [==============================] - 467s 5s/step - loss: 0.5167 - acc: 0.7360 - val_loss: 0.3032 - val_acc: 0.8860
Epoch 2/30
100/100 [==============================] - 465s 5s/step - loss: 0.4070 - acc: 0.8230 - val_loss: 0.2364 - val_acc: 0.8960
Epoch 3/30
100/100 [==============================] - 464s 5s/step - loss: 0.3371 - acc: 0.8530 - val_loss: 0.3038 - val_acc: 0.9050
Epoch 4/30
100/100 [==============================] - 456s 5s/step - loss: 0.3248 - acc: 0.8695 - val_loss: 0.4519 - val_acc: 0.8280
Epoch 5/30
100/100 [==============================] - 457s 5s/step - loss: 0.3082 - acc: 0.8710 - val_loss: 0.2467 - val_acc: 0.8870
Epoch 6/30
100/100 [==============================] - 456s 5s/step - loss: 0.2728 - acc: 0.8845 - val_loss: 0.3176 - val_acc: 0.8960
Epoch 7/30
100/100 [==============================] - 456s 5s/step - loss: 0.3221 - acc: 0.8685 - val_loss: 0.2944 - val_acc: 0.8810
Epoch 8/30
100/100 [==============================] - 456s 5s/step - 

Fine-tunning the model

In [9]:
# Freezing all layers up to a specific one

conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [10]:
# fine tunning by setting a low learning rate and adding 100 epochs

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/100
100/100 [==============================] - 230s 2s/step - loss: 0.1652 - acc: 0.9295 - val_loss: 0.1611 - val_acc: 0.9430
Epoch 2/100
100/100 [==============================] - 230s 2s/step - loss: 0.1312 - acc: 0.9500 - val_loss: 0.2975 - val_acc: 0.9250
Epoch 3/100
100/100 [==============================] - 230s 2s/step - loss: 0.1363 - acc: 0.9515 - val_loss: 0.2368 - val_acc: 0.9380
Epoch 4/100
100/100 [==============================] - 228s 2s/step - loss: 0.1284 - acc: 0.9460 - val_loss: 0.2934 - val_acc: 0.9380
Epoch 5/100
100/100 [==============================] - 229s 2s/step - loss: 0.1404 - acc: 0.9445 - val_loss: 0.2082 - val_acc: 0.9410
Epoch 6/100
100/100 [==============================] - 230s 2s/step - loss: 0.1442 - acc: 0.9495 - val_loss: 0.2365 - val_acc: 0.9360
Epoch 7/100
100/100 [==============================] - 227s 2s/step - loss: 0.1220 - acc: 0.9485 - val_loss: 0.3915 - val_acc: 0.9240
Epoch 8/100
100/100 [==============================] - 227s 2s

Epoch 62/100
100/100 [==============================] - 225s 2s/step - loss: 0.0871 - acc: 0.9710 - val_loss: 0.3914 - val_acc: 0.9340
Epoch 63/100
100/100 [==============================] - 225s 2s/step - loss: 0.1008 - acc: 0.9675 - val_loss: 0.2666 - val_acc: 0.9510
Epoch 64/100
100/100 [==============================] - 224s 2s/step - loss: 0.0792 - acc: 0.9760 - val_loss: 0.3175 - val_acc: 0.9450
Epoch 65/100
100/100 [==============================] - 226s 2s/step - loss: 0.1124 - acc: 0.9665 - val_loss: 0.3271 - val_acc: 0.9350
Epoch 66/100
100/100 [==============================] - 225s 2s/step - loss: 0.0829 - acc: 0.9670 - val_loss: 0.2903 - val_acc: 0.9480
Epoch 67/100
100/100 [==============================] - 225s 2s/step - loss: 0.0830 - acc: 0.9705 - val_loss: 0.2750 - val_acc: 0.9420
Epoch 68/100
100/100 [==============================] - 225s 2s/step - loss: 0.0747 - acc: 0.9745 - val_loss: 0.3571 - val_acc: 0.9320
Epoch 69/100
100/100 [==============================] -

Plotting the results

In [12]:
# evaluating the model

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

Found 1000 images belonging to 2 classes.
test acc: 0.9409999942779541
